In [1]:
import pandas as pd
import numpy as np

In [ ]:
# products_sheet_id = "1FB2bbvWE2nsf49KO_DkWzbL2hm9r4G8j"
# products_list = "Tablib%20Dataset"
# url = f"https://docs.google.com/spreadsheets/d/{products_sheet_id}/gviz/tq?tqx=out:csv&sheet={products_list}"

In [2]:
pdt_df = pd.read_csv('full_products_list.csv')
batches_df = pd.read_csv('batches_list.csv')
recipe_df = pd.read_csv('recipe_list.csv')

In [48]:
pdt_df.head()

,ID,COUNTRY OF ORIGIN,FORMULATION,RETAIL PRICE PER SMALLEST UNIT,RETAIL PRICE PER SMALLEST UNIT.1,PACK SIZE,GENERIC NAME,BRAND NAME,STRENGTH,eshop_id
0,897,India,Tablet,100.0,10000.0,100,METRONIDAZOLE,AARTROGYL 200,200mg,NaN
1,241,Uganda,Solution,3000.0,3000.0,1,Gentamycin Sulphate,ABGENTA GENTAMICIN EAR/EYE DROPS,0.3%w/v,8272.0
2,5497,NaN,Solution,10000.0,10000.0,0,Timolol,ABLOL EYE DROPS 0.5%,0.5%,NaN
3,243,Uganda,Solution,5000.0,5000.0,1,"Sodium chloride, Benzalkonium chloride",ABNAL NASAL DROPS,0.9%/0.01%w/v,8666.0
4,635,Uganda,Injection,10000.0,10000.0,1,Paracetamol,ABPARA INJ 100ML,1%,NaN


In [57]:
pdt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2604 entries, 0 to 2603
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                2604 non-null   int64  
 1   COUNTRY OF ORIGIN                 1239 non-null   object 
 2   FORMULATION                       1514 non-null   object 
 3   RETAIL PRICE PER SMALLEST UNIT    2576 non-null   float64
 4   RETAIL PRICE PER SMALLEST UNIT.1  2526 non-null   float64
 5   PACK SIZE                         2604 non-null   int64  
 6   GENERIC NAME                      2604 non-null   object 
 7   BRAND NAME                        2604 non-null   object 
 8   STRENGTH                          2267 non-null   object 
 9   eshop_id                          394 non-null    float64
dtypes: float64(3), int64(2), object(5)
memory usage: 203.6+ KB


In [61]:
for prod in pdt_df['GENERIC NAME']:
    for p in unique_pdts_df['GENERIC NAME']:
        if np.where(pdt_df['GENERIC NAME'].unique() == unique_pdts_df['GENERIC NAME'].unique):
            print(prod)

METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
METRONIDAZOLE
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sulphate
Gentamycin Sul

Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Acnes
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream Cleanser
Cream C

Triprolidine hydrochloride/Pseudoephedrine hydrochloride
Triprolidine hydrochloride/Pseudoephedrine hydrochloride
Triprolidine hydrochloride/Pseudoephedrine hydrochloride
Triprolidine hydrochloride/Pseudoephedrine hydrochloride
Triprolidine hydrochloride/Pseudoephedrine hydrochloride
Triprolidine hydrochloride/Pseudoephedrine hydrochloride
Triprolidine hydrochloride/Pseudoephedrine hydrochloride
Triprolidine hydrochloride/Pseudoephedrine hydrochloride
Triprolidine hydrochloride/Pseudoephedrine hydrochloride
Triprolidine hydrochloride/Pseudoephedrine hydrochloride
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphenesin
Pseudoephedrine,Guaiphen

Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylprednisolone
Methylpredni

Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendronic Acid
Alendron

...

Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albendazole
Albe

Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin 

Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Celecoxib 100mg
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine
Cetirizine

Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 10mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Tabs 20mg
Lisinopril Ta

Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Caps
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromycin Susp
Azithromyc

In [ ]:
# batches_df.head()

In [ ]:
# recipe_df.head()

In [ ]:
# pdt_df.columns

In [3]:
new_pdt_df= pdt_df.drop(['COUNTRY OF ORIGIN', 'FORMULATION', 'PACK SIZE', 'RETAIL PRICE PER SMALLEST UNIT', 
             'RETAIL PRICE PER SMALLEST UNIT.1', 'PACK SIZE', 'STRENGTH', 'eshop_id'], axis=1)

In [ ]:
# new_pdt_df.dtypes

In [ ]:
# new_pdt_df.info()

In [ ]:
# new_pdt_df.head()

In [ ]:
# batches_df.columns

In [4]:
new_batches_df= batches_df.drop(['id', 'created', 'modified', 'initial_item_count', 
             'cost_price', 'item_count', 'expiration_date', 'manufacturing_date'], axis=1)

In [ ]:
# new_batches_df.info()

In [ ]:
# recipe_df.columns

In [5]:
new_recipe_df = recipe_df.drop(['id', 'created', 'modified', 'prescription', 'dosage', 'tablets', 'quantity', 'frequency', 
                                'duration', 'route', 'price_discount', 'serving_pharmacy', 'price_controller', 'rx_price',
                               'external_price', 'requires_approval', 'approved', 'dropped', 'canceling_party', 
                                'reason_for_cancelation', 'authorized_by_tmcg', 'authorized_by_insurer', 'client_decision', 
                                'insurer_code'], axis=1)

In [ ]:
# new_recipe_df.info()

In [ ]:
# new_recipe_df.head()

In [6]:
xls = pd.ExcelFile(r'C:\Users\NDA\Desktop\EVE\TMCG\documents\duplicate_products.xlsx')

In [7]:
dp_list = pd.read_excel(xls, 'copy_final_list_duplicates')
unique_pdts_df = pd.read_excel(xls, 'unique final list')

In [54]:
dp_list.head()

,Eva,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,3951,NaN,NaN,25000,25000,0,Betadine Mouthwash Gargle,Betadine Mouthwash Gargle,125ml,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,387,Germany,Tablet,1500,42000,28,Montelukast,DENK AIR JUNIOR 5mg,5mg,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4933,NaN,NaN,2200,2200,28,Montelukast Tabs,Denk-air,10mg,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4942,NaN,NaN,1500,1500,28,Montelukast Tabs,Denk-Air Junior,5mg,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3146,NaN,NaN,300,300,100,Diclofenac Tabs,Diclomol SR,100mg,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
unique_pdts_df.head()

,PRODUCT ID,GENERIC NAME,BRAND NAME
0,5302,Glyceryl trinitrate,accord 500mcg
1,3949,Betadine Antiseptic Solution,Betadine Antiseptic Solution
2,715,Montelukast,DENK AIR 10MG
3,4328,Diclofenac SR Tabs,Diclomol
4,830,Doxazosin,DOXAZOSIN 4MG UK


In [ ]:
# unique_pdts_df.info()

In [ ]:
# dp_list.columns

In [ ]:
# dp_list.info()

In [8]:
new_dp_list = dp_list.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 8',
                           'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
                           'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
                           'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23'], axis=1)

In [9]:
new_dp_list.head()

,Eva,Unnamed: 6,Unnamed: 7
0,3951,Betadine Mouthwash Gargle,Betadine Mouthwash Gargle
1,387,Montelukast,DENK AIR JUNIOR 5mg
2,4933,Montelukast Tabs,Denk-air
3,4942,Montelukast Tabs,Denk-Air Junior
4,3146,Diclofenac Tabs,Diclomol SR


In [10]:
new_dp_list = new_dp_list.rename(columns={'Eva': 'PRODUCT ID', 'Unnamed: 6': 'Generic Name', 'Unnamed: 7': 'Brand Name'})

In [11]:
new_dp_list.head()

,PRODUCT ID,Generic Name,Brand Name
0,3951,Betadine Mouthwash Gargle,Betadine Mouthwash Gargle
1,387,Montelukast,DENK AIR JUNIOR 5mg
2,4933,Montelukast Tabs,Denk-air
3,4942,Montelukast Tabs,Denk-Air Junior
4,3146,Diclofenac Tabs,Diclomol SR


In [ ]:
new_dp_list.info()

In [12]:
new_dp_list = new_dp_list.dropna(axis='rows')

In [13]:
unique_pdts_df.columns

Index(['ID', 'COUNTRY OF ORIGIN', 'FORMULATION',
       'RETAIL PRICE PER SMALLEST UNIT', 'RETAIL PRICE PER SMALLEST UNIT.1',
       'PACK SIZE', 'GENERIC NAME', 'BRAND NAME', 'STRENGTH', 'eshop_id',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')

In [14]:
unique_pdts_df = unique_pdts_df.drop(['COUNTRY OF ORIGIN', 'FORMULATION', 'RETAIL PRICE PER SMALLEST UNIT', 
                                      'RETAIL PRICE PER SMALLEST UNIT.1', 'PACK SIZE', 'STRENGTH', 'eshop_id', 
                                      'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 
                                      'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 
                                      'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 
                                      'Unnamed: 25'], axis=1)

In [56]:
# unique_pdts_df.head(49)

In [ ]:
# new_dp_list['PRODUCT ID'].to_list()

In [ ]:
new_dp_list.tail()

In [15]:
new_dp_list = new_dp_list.drop([new_dp_list.index[-1]])

In [16]:
new_dp_list.tail()

,PRODUCT ID,Generic Name,Brand Name
50,5379,N'RGY + PLUS 30'S,N'RGY + PLUS 30'S
51,3814,Amlodipine Tabs,Norvasc 5
52,538,Cetylpyridinium Chloride and Lidocaine,ORACURE GEL 15G
53,636,Promethazine,PROMETHAZINE INJ
54,5394,Felodipine SR Tabs,Vascalpha


In [17]:
new_dp_list['PRODUCT ID'] = pd.to_numeric(new_dp_list['PRODUCT ID'], errors='ignore')

In [ ]:
new_dp_list.info()

In [ ]:
# new_dp_list['PRODUCT ID'].to_list()

In [18]:
new_pdt_df = new_pdt_df.rename(columns={'ID': 'PRODUCT ID'})

In [19]:
new_pdt_df.head()

,PRODUCT ID,GENERIC NAME,BRAND NAME
0,897,METRONIDAZOLE,AARTROGYL 200
1,241,Gentamycin Sulphate,ABGENTA GENTAMICIN EAR/EYE DROPS
2,5497,Timolol,ABLOL EYE DROPS 0.5%
3,243,"Sodium chloride, Benzalkonium chloride",ABNAL NASAL DROPS
4,635,Paracetamol,ABPARA INJ 100ML


In [20]:
merged_pdt_dup = pd.merge(new_dp_list, new_pdt_df, on=['PRODUCT ID'], how='inner')

In [ ]:
# merged_pdt_dup.head(51)

In [21]:
merged_batch_dup = pd.merge(new_dp_list, new_batches_df, on=['PRODUCT ID'], how='inner')

In [ ]:
merged_batch_dup.info()

In [ ]:
new_dp_list.shape

In [ ]:
# new_recipe_df

In [22]:
new_recipe_df = new_recipe_df.rename(columns={'product_id': 'PRODUCT ID'})

In [23]:
new_recipe_df.head()

,PRODUCT ID,product
0,807,VALUPACK VITAMIN C (Vitamin C)
1,642,LEVOBACT 750MG (Levofloxacin)
2,334,FOLIC ACID (Generic) (Folic Acid)
3,537,LUCIN CREAM 15G (Hydrocortisone)
4,225,PANTOP-D (Pantoprazole/Domperidone)


In [24]:
merged_recipe_dup = pd.merge(new_dp_list, new_recipe_df, on=['PRODUCT ID'], how='inner')

In [25]:
merged_recipe_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473 entries, 0 to 472
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PRODUCT ID    473 non-null    int64 
 1   Generic Name  473 non-null    object
 2   Brand Name    473 non-null    object
 3   product       473 non-null    object
dtypes: int64(1), object(3)
memory usage: 18.5+ KB


In [ ]:
# new_pdt_df
# new_batches_df
# new_recipe_df

# merged_pdt_dup
# merged_batch_dup
# merged_recipe_dup

In [26]:
unique_pdts_df = unique_pdts_df.rename(columns={'ID': 'PRODUCT ID'})

In [28]:
unique_pdts_df.head()

,PRODUCT ID,GENERIC NAME,BRAND NAME
0,5302,Glyceryl trinitrate,accord 500mcg
1,3949,Betadine Antiseptic Solution,Betadine Antiseptic Solution
2,715,Montelukast,DENK AIR 10MG
3,4328,Diclofenac SR Tabs,Diclomol
4,830,Doxazosin,DOXAZOSIN 4MG UK


In [27]:
merged_batch_dup.head()

,PRODUCT ID,Generic Name,Brand Name,batch_number,product__generic_name,product__brand_name
0,387,Montelukast,DENK AIR JUNIOR 5mg,23260,Montelukast,DENK AIR JUNIOR 5mg
1,387,Montelukast,DENK AIR JUNIOR 5mg,21527,Montelukast,DENK AIR JUNIOR 5mg
2,5416,Levothyroxine,EUTHYROX 100MCG 30 Tablet,G00LR6,Levothyroxine,EUTHYROX 100MCG TAB
3,5416,Levothyroxine,EUTHYROX 100MCG 30 Tablet,123AB,Levothyroxine,EUTHYROX 100MCG TAB
4,3819,"Amlodipine besilate,Valsatan",EXFORGE 5/160MG,BLE95,"Amlodipine besilate,Valsatan",EXFORGE 5/160MG


In [ ]:
new_batch = merged_batch_dup.replace()

In [34]:
# for prod in merged_batch_dup['Generic Name']:
#     if np.where(merged_batch_dup['Generic Name'] == unique_pdts_df['GENERIC NAME'], True, False):
#         print(prod)

In [35]:
merged_series = merged_batch_dup['Generic Name']
unique_series = unique_pdts_df['GENERIC NAME']

In [38]:
merged_batch_dup.columns

Index(['PRODUCT ID', 'Generic Name', 'Brand Name', 'batch_number',
       'product__generic_name', 'product__brand_name'],
      dtype='object')

In [40]:
# merged_row = merged_batch_dup.loc[['Montelukast']]
# merged_row

In [45]:
merged_batch_dup['Generic Name'].unique()

array(['Montelukast', 'Levothyroxine', 'Amlodipine besilate,Valsatan',
       'Amlodipine/ Valsartan', 'Face Shields',
       'Kabuuti Herbal Cough Syrup', 'Metronidazole', 'Desmopressin',
       'Loratidine', 'Fluoxetine Caps', 'Azithromycin Susp',
       'Desloratidine', 'Clopidogrel', 'Diclofenac 12.5mg Suppositories',
       'Face Mask', 'Captopril Tabs', 'FLUCONAZOLE', 'Losartan potassium',
       "N'RGY + PLUS 30'S", 'Cetylpyridinium Chloride and Lidocaine',
       'Promethazine', 'Felodipine SR Tabs'], dtype=object)

In [46]:
unique_pdts_df['GENERIC NAME'].unique()

array(['Glyceryl trinitrate', 'Betadine Antiseptic Solution',
       'Montelukast', 'Diclofenac SR Tabs', 'Doxazosin',
       'Duo Cortexin Adult', 'Dihdroartemisinine,Piperaquine',
       'Diclofenac Injection', 'Dynapar Injection', 'E nat caps',
       'Mometasone', 'Vitamin E', 'Crotamiton Cream',
       'Levothyroxine Tabs', 'Amlodipine besilate,Valsartan',
       'Amlodipine/ Valsartan', 'Face sheild', 'Ketoprofen Gel',
       'Levonorgestrel', 'Herbal Cough Mixture', 'Lamotrigine',
       'Metronidazole', 'Desmopressin', 'Loratadine', 'Fluoxetine',
       'Metformin', 'Azithromycin', 'Desloratidine 5mg',
       'Atorvastatin 20mg tabs', 'Beechams Max strength ALL IN ONE Caps',
       'Betadine Mouthwash Gargle', 'Hyoscine Butylbromide Syp',
       'Montelukast+Levocetrizine Tabs', 'Clopidogrel', 'Caustic Pencil',
       'Ciprofloxacin Tabs', 'Diclofenac Supp', 'Face Mask',
       'Captopril Tabs', 'FLUCONAZOLE', 'Losartan',
       'Miconazole, Clobestol, Gentamycin', 'Methotrexat

In [47]:
for prod in merged_batch_dup['Generic Name']:
    if np.where(merged_batch_dup['Generic Name'].unique() == unique_pdts_df['GENERIC NAME'].unique):
        print(prod)

Montelukast
Montelukast
Levothyroxine
Levothyroxine
Amlodipine besilate,Valsatan
Amlodipine besilate,Valsatan
Amlodipine besilate,Valsatan
Amlodipine besilate,Valsatan
Amlodipine besilate,Valsatan
Amlodipine besilate,Valsatan
Amlodipine besilate,Valsatan
Amlodipine/ Valsartan
Face Shields
Face Shields
Kabuuti Herbal Cough Syrup
Kabuuti Herbal Cough Syrup
Metronidazole
Metronidazole
Desmopressin
Desmopressin
Loratidine
Loratidine
Loratidine
Loratidine
Loratidine
Loratidine
Loratidine
Loratidine
Loratidine
Loratidine
Loratidine
Loratidine
Fluoxetine Caps
Fluoxetine Caps
Fluoxetine Caps
Fluoxetine Caps
Azithromycin Susp
Desloratidine
Desloratidine
Desloratidine
Desloratidine
Desloratidine
Desloratidine
Desloratidine
Desloratidine
Desloratidine
Clopidogrel
Clopidogrel
Clopidogrel
Diclofenac 12.5mg Suppositories
Face Mask
Face Mask
Face Mask
Face Mask
Face Mask
Face Mask
Face Mask
Face Mask
Face Mask
Face Mask
Face Mask
Captopril Tabs
FLUCONAZOLE
FLUCONAZOLE
Losartan potassium
N'RGY + PLUS 

In [ ]:
unique_pdts_df.loc()

In [55]:
pdt_df[pdt_df['GENERIC NAME'] == 'Montelukast']

,ID,COUNTRY OF ORIGIN,FORMULATION,RETAIL PRICE PER SMALLEST UNIT,RETAIL PRICE PER SMALLEST UNIT.1,PACK SIZE,GENERIC NAME,BRAND NAME,STRENGTH,eshop_id
424,4936,India,Tablet,700.0,700.0,30,Montelukast,BREATHEZY 4MG TAB,4mg,NaN
425,4939,India,Tablet,800.0,800.0,30,Montelukast,BREATHEZY CW 5MG TAB,5mg,NaN
757,715,Germany,Tablet,3800.0,106400.0,28,Montelukast,DENK AIR 10MG,10g,NaN
758,383,Germany,Tablet,1700.0,47600.0,28,Montelukast,DENK AIR JUNIOR 4MG,4mg,6877.0
759,387,Germany,Tablet,1500.0,42000.0,28,Montelukast,DENK AIR JUNIOR 5mg,5mg,NaN
1654,720,India,Tablet,500.0,5000.0,10,Montelukast,MARTINEZ10,10mg,NaN
1712,254,UK,Tablet,1000.0,28000.0,28,Montelukast,MONTELUKAST 1OMG UK TEVA,10mg,6196.0
1713,415,UK,Tablet,1500.0,28.0,42000,Montelukast,MONTELUKAST BRISTOL 10mg,10mg,NaN
1714,716,India,Tablet,800.0,24000.0,30,Montelukast,MONTIRIV 10MG,10mg,NaN
2415,676,Jordan,Tablet,800.0,24000.0,30,Montelukast,UNICAST 10MG,10mg,NaN


In [ ]:
unique_pdts_df[unique_pdts_df['GENERIC NAME'] == 'Montelukast']